This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras import regularizers

In [2]:
BASE_DIR = "/content/muffin-vs-chihuahua"
train_dir = os.path.join(BASE_DIR, "train")
test_dir = os.path.join(BASE_DIR, "test")

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(
        32, (3, 3), activation='relu',
        kernel_regularizer=regularizers.l2(0.001),
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    ),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(
        64, (3, 3), activation='relu',
        kernel_regularizer=regularizers.l2(0.001)
    ),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(
        128, (3, 3), activation='relu',
        kernel_regularizer=regularizers.l2(0.001)
    ),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(
        128, activation='relu',
        kernel_regularizer=regularizers.l2(0.001)
    ),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.6128 - loss: 0.9147 - val_accuracy: 0.8233 - val_loss: 0.5252
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.8086 - loss: 0.5356 - val_accuracy: 0.8434 - val_loss: 0.4417
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.8402 - loss: 0.4492 - val_accuracy: 0.8296 - val_loss: 0.4372
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 145s 1s/step - accuracy: 0.8405 - loss: 0.4595 - val_accuracy: 0.8878 - val_loss: 0.3571
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.8473 - loss: 0.4145 - val_accuracy: 0.8889 - val_loss: 0.3569
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.8582 - loss: 0.4157 - val_accuracy: 0.8984 - val_loss: 0.3515
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 194s 1s/step - accuracy: 0.8688 - loss: 0.3754 - val_accuracy: 0.9153 - val_loss: 0.3056
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.8755 - loss: 0.3725 - val_accu

In [8]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 14s 372ms/step - accuracy: 0.9084 - loss: 0.3162
Test Accuracy: 0.9096283912658691


In [21]:
# SAVE THE MODEL
model.save("exercise_6_trained_model_improved.h5")

In [10]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [20]:
images, labels = next(test_generator)
pred = model.predict(images)

confidence = pred[0][0] * 100
label = "Muffin" if pred[0][0] > 0.5 else "Chihuahua"

print("Prediction:", label)
print(f"Confidence: {confidence:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
Prediction: Chihuahua
Confidence: 17.24%


Run 1:
Prediction: Chihuahua
Confidence: 17.29%

Run 2:
Prediction: Chihuahua
Confidence: 17.24%